<h1>Report 1<h1/>

In this notebook I am going to present the code for the resolution of the Assessment 1 in the module "Programming and Policy Analysis".

<h2>Loading Database<h2/>

In this section I am going to upload the database. It could be any database, but in this case I will use the database "Student Performance" from an Introductory Paper for 5th Annual Future Business Technology Conference.

In [1]:
!pip3 install -U ucimlrepo 

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
student_performance = fetch_ucirepo(id=320) 
  
# data (as pandas dataframes) 
X = student_performance.data.features 
y = student_performance.data.targets 
  
# metadata 
print(student_performance.metadata) 
  
# variable information 
print(student_performance.variables) 

print(y)
print (X)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


{'uci_id': 320, 'name': 'Student Performance', 'repository_url': 'https://archive.ics.uci.edu/dataset/320/student+performance', 'data_url': 'https://archive.ics.uci.edu/static/public/320/data.csv', 'abstract': 'Predict student performance in secondary education (high school). ', 'area': 'Social Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 649, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': ['Sex', 'Age', 'Other', 'Education Level', 'Occupation'], 'target_col': ['G1', 'G2', 'G3'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5TG7T', 'creators': ['Paulo Cortez'], 'intro_paper': {'ID': 360, 'type': 'NATIVE', 'title': 'Using data mining to predict secondary school student performance', 'authors': 'P. Cortez, A. M. G. Silva', 'venue': 'Proceedings of 5th Annual Future Business Technolo

<h2>Estimation of Parameters<h2/>

In this section I am going to estimate the parameters of the OLS. The first step is to create an unitary matrix.

In [2]:
matrix_one=[]

for f in range(len(y)):
    matrix_one.append([])
    for c in range(1):
        matrix_one[f].append(1)


print(matrix_one)

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1],

Next, I am going to create a matrix of independent variables that will be united with the unitary matrix. Furthermore, it will be created the matrix of dependent variable.

In [4]:
# Import some libraries for working in a better way

import pandas as pd
import numpy as np


df=pd.DataFrame(X)
dg=pd.DataFrame(y)

# Matrix of independent variables
independent=df[['age','traveltime','studytime','famrel','freetime','absences']].to_numpy()

# Matrix of dependent variable
dependent=dg[['G3']].to_numpy()

# Unitary matrix is transformed in an array
matrix_one=np.array(matrix_one)

# Unitary matrix is united with independent variables
explicative=np.hstack((matrix_one,independent))

# Matrix of independent variables (united to unitary matrix) is transposed
x_t=np.transpose(explicative)

# Estimation of betas
betas=np.linalg.inv(x_t@explicative)@x_t@dependent
print(betas)

[[15.35557546]
 [-0.2462774 ]
 [-0.47082367]
 [ 0.89095476]
 [ 0.24453112]
 [-0.36127202]
 [-0.03270453]]


<h2>Standard Errors<h2/>

In this section it will be estimated the Standard Errors

In [6]:
# It is calculated the residuals 
residuos=dependent-explicative@betas

# It is calculated the variance of the errors
var_errors=(np.transpose(residuos)@residuos)*(1/(len(residuos)-explicative.shape[1]))

#It is calculated the Var-Cov matrix
var_cov=var_errors*np.linalg.inv(x_t@explicative)

#It is calculated the standard errors
standard_errors=np.sqrt(np.diag(var_cov))
print(standard_errors)

[1.81565357 0.10027579 0.16165395 0.1472092  0.12784372 0.11607241
 0.02660215]


<h2>t-Statistics<h2/>

In this section I am going to calculate the t statistics associated with each beta.

In [8]:
# The shape of standard errors is changed for allowing the calculation
standard_errors1=standard_errors.reshape(7,1)

# The t-statistics are calculated for each beta
t_statistics=betas*np.reciprocal(standard_errors1)

print(t_statistics)

[[ 8.4573267 ]
 [-2.45600047]
 [-2.91254036]
 [ 6.05230352]
 [ 1.91273474]
 [-3.11247118]
 [-1.22939442]]


<h2>p-Values<h2/>

In this section I am going to calculate the p values associated to each beta.

In [9]:
# Import this library for getting the cumulative distribution function
from scipy import stats

#p-values are estimated
p_value=2*(1-stats.t.cdf(abs(t_statistics),len(residuos)-explicative.shape[1]))
print(p_value)

[[2.22044605e-16]
 [1.43131173e-02]
 [3.70955924e-03]
 [2.42799025e-09]
 [5.62266489e-02]
 [1.93769711e-03]
 [2.19374335e-01]]


<h2>R-Square<h2/>

In this section I am going to calculate de R-Square of this regression.

In [11]:
# Average of the dependent variable
y_bar=np.sum(dependent)/len(dependent)

# Total Sum of Squares (SST)
SST=np.sum((dependent-y_bar)**2)

# Dependent variable estimated
y_hat=explicative@betas

# Residual Sum of Squares
SSE=np.sum((dependent-y_hat)**2)

# R-Square is calculated
R_square=1-(SSE/SST)
print(R_square)

0.10420715318150897
